## Well-known graph traversal implimentations (simplified):

#### 1- Djikstra Algorithm to find the minimum shortest path. O(V**2)
#### 2- Priority Heap 
#### 3- Djikstra Algorithm improved with the priority heap so that the running time is O(V*logE)  
#### 4- Bellman Ford Algorithm to find the minimum shortest path. 

In [1]:
# prepare a densely connected test matrix with random numbers
# to represent a densely connected directed cyclic graph, initialize the distances as 0 from each vertex to itself

import numpy as np
np.random.seed(0)
def test_graph(n): # n vertices
    test_graph_set = np.random.rand(n,n)*100
    # 0 the source so that the node is acyclic to itself 
    for i in range(n):
        test_graph_set[i][i] = np.inf
        # test_graph_set[i][:i] = np.inf  # unmark if a tree structure is needed
    test_graph_set = np.around(test_graph_set, decimals = 2)
    return test_graph_set

test_set = test_graph(10) 
print(test_set[:10][:10])



[[  inf 71.52 60.28 54.49 42.37 64.59 43.76 89.18 96.37 38.34]
 [79.17   inf 56.8  92.56  7.1   8.71  2.02 83.26 77.82 87.  ]
 [97.86 79.92   inf 78.05 11.83 63.99 14.34 94.47 52.18 41.47]
 [26.46 77.42 45.62   inf  1.88 61.76 61.21 61.69 94.37 68.18]
 [35.95 43.7  69.76  6.02   inf 67.06 21.04 12.89 31.54 36.37]
 [57.02 43.86 98.84 10.2  20.89   inf 65.31 25.33 46.63 24.44]
 [15.9  11.04 65.63 13.82 19.66 36.87   inf  9.71 83.79  9.61]
 [97.65 46.87 97.68 60.48 73.93  3.92 28.28   inf 29.61 11.87]
 [31.8  41.43  6.41 69.25 56.66 26.54 52.32  9.39   inf 92.93]
 [31.86 66.74 13.18 71.63 28.94 18.32 58.65  2.01 82.89   inf]]


### Naive Djikstra

In [2]:
# write a naive DIJKSTRA's algorithm to find the shortest distance from a source vertex to all other vertices (O(v**2))
def graph_traverse_DJ_path(graph, source): 
    distance_list = [np.inf] * len(graph)
    visited_vertices_list = [False] * len(graph)
    distance_list[source] = 0
    for each_vertex in range(len(graph)):
        min_value = np.inf
        for i in range(len(distance_list)):
            if distance_list[i] < min_value and visited_vertices_list[i] == False:
                min_value = distance_list[i]
                min_at_index = i
        visited_vertices_list[min_at_index] = True
        for i in range(len(distance_list)):
            if visited_vertices_list[i] == False and graph[min_at_index][i] > 0 and \
            distance_list[min_at_index] + graph[min_at_index][i] < distance_list[i]: 
                distance_list[i] = distance_list[min_at_index] + graph[min_at_index][i]
    return np.around(distance_list, decimals = 2)

print(graph_traverse_DJ_path(test_set, 0))
%timeit graph_traverse_DJ_path(test_set, 0)

[ 0.   54.8  51.52 48.39 42.37 44.27 43.76 40.35 69.96 38.34]
95.7 µs ± 6.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
# visualize the parent - child nodes' index relationship in a heap:
nodes_indices = np.arange(0,5)
index = 0
right_child = 0
while index < len(nodes_indices):
    left_child = index *2 + 1
    right_child = index *2 + 2
    print('parent index:', index, 'left child index:', left_child, 'right child index:', right_child)
    index += 1

# percolate up: while index i > 0: move from last index to top and switch the parent and left child, update i.
# percolate down: while index i < len//2: find min child of that node, switch the parent and mc, update i as mc 

parent index: 0 left child index: 1 right child index: 2
parent index: 1 left child index: 3 right child index: 4
parent index: 2 left child index: 5 right child index: 6
parent index: 3 left child index: 7 right child index: 8
parent index: 4 left child index: 9 right child index: 10


### Priority Heap (minimum at first index)

In [4]:
# create a heapq to later use it in the djikstra's algorithm
class Heapq:
    def __init__(self):
        self.heap = []
    
            
    def push(self,k): # append k to the array and percolate up with the last index
        self.heap.append(k)
        index_of_last_item = len(self.heap)-1 
        self.percUp(index_of_last_item) # last item's index

             
    def percUp(self,i): # commonly known as "siftdown method"
        while i > 0: # smallest index need to be 1 since first index is 0
            if self.heap[i] < self.heap[(i-1) // 2]: # if child is smaller than its parent, swap
                tmp = self.heap[(i-1) // 2]
                self.heap[(i-1) // 2] = self.heap[i]
                self.heap[i] = tmp
            i = (i-1) // 2

            
    def pop(self):
        if len(self.heap) == 0: return None
        if len(self.heap) == 1: 
            tmp = self.heap.pop()
            return tmp
        retval = self.heap[0] # first item
        last_item = self.heap.pop() # delete last item and assign it to a variable
        self.heap[0] = last_item  # swap first item with last item
        self.percDown(0)
        return retval


    def percDown(self,i): # commonly known as "siftup method"
        # leftchild index is 2*i + 1, rightchild index is 2*i + 2, 
        # rightchild might not exist
        while i < (len(self.heap) // 2):
            mc = self.minChild(i)
            if self.heap[i] > self.heap[mc]:
                tmp = self.heap[i]
                self.heap[i] = self.heap[mc]
                self.heap[mc] = tmp
            i = mc   
            
            
    def minChild(self,i):
        left_child_index = (i*2) + 1
        right_child_index = left_child_index + 1
        # if there is no right child, return the left child's index
        if (left_child_index + 1) == len(self.heap): return left_child_index
        elif self.heap[left_child_index] < self.heap[right_child_index]: # choose the smaller child 
            return left_child_index
        else:
            return right_child_index


    def heapify(self,alist): 
        self.heap = alist[:]
        n = len(self.heap)
        for i in reversed(range(n//2)): # 4, 3, 2, 1, 0 and 0 is also included.
            self.percDown(i)
        return self.heap
    
    
         
if __name__== "__main__":
            
    bh_test = Heapq()
    test_set2 = [24,26,6,27,725,3,5245,245,78,8,8,0,9] 
    bh_test.heapify(test_set2)
    print(bh_test.heap)
    test_set3 = [[0.0, 'hello'], [10.02, 'first'], [97.87, 'bear'], [224.09, 'dog'], [186.76, 'oink'], [10.32, '0']]
    bh_test.heapify(test_set3)
    print(bh_test.heap)
    value, text = bh_test.pop()
    print(value, type(value), text, type(text))

[0, 8, 3, 27, 8, 6, 5245, 245, 78, 26, 725, 24, 9]
[[0.0, 'hello'], [10.02, 'first'], [10.32, '0'], [224.09, 'dog'], [186.76, 'oink'], [97.87, 'bear']]
0.0 <class 'float'> hello <class 'str'>


### Compare the heap with the built-in heap and a python built-in quick sort to see its efficiency

In [5]:
# check the efficiency of the heap above and compare your heapq above with a python's built in module
import heapq # python built-in heap 
import numpy as np
np.random.seed(0)
testing_list = list(np.random.randint(1,1000, size = 500))

# my_heap object below takes the list and heapifies it and outputs the object as priority heap:
my_heap = Heapq()
%timeit my_heap.heapify(testing_list)
print(my_heap.heap[:20])

# using python built-in heap.heapify to convert a list into priority heap in place
python_heap_sorted = heapq.heapify(testing_list) 
%timeit heapq.heapify(testing_list) 
print(testing_list[:20])

# Additional: The python built-in sorting nlogn algorithm to sort the whole list
quick_sorted = np.sort(list(np.random.randint(1,1000, size = 500)))
%timeit np.sort(list(np.random.randint(1,1000, size = 500)))
print(quick_sorted[:20])

# Conclusion: The heapq class written above is working very slowly. Python built-in heapq class is the fastest.

546 µs ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
[5, 10, 14, 12, 12, 17, 26, 25, 24, 34, 30, 20, 48, 111, 42, 29, 33, 149, 43, 45]
29.3 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[5, 10, 14, 12, 12, 17, 26, 25, 24, 34, 30, 20, 48, 111, 42, 29, 33, 149, 43, 45]
125 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[ 1  1  4  4  5  8 13 14 18 21 22 23 25 27 29 30 33 36 43 47]


### Improved Djikstra with Heapq 

In [6]:
# Keep the distances in a dictionary where the keys are the name of vertices and the values are the total 
# minimum distances traversed from source node to other nodes

import numpy as np
import heapq

def djikstra_with_heapq(graph, starting_vertex):
    distances = {vertex: np.inf for vertex in range(len(graph))} # {'0': inf, '1': inf, '2': inf,...}
    distances[starting_vertex] = 0 # {'0': 0, '1': inf, '2': inf,...} it is made acyclic
    visited_vertices_dict = [False] * len(graph) # {'0': 'False', '1': 'False',..} to make sure each vertex is visited once
    # loop over vertices
    for k in distances.keys(): # for each vertex in hash_table, append the connection list to heapq
        # find the min distance by heapifying the distances list, 
        # the values and keys need to switch places since values are going to be compared and sorted
        distances_as_heap = [[value, key] for key, value in distances.items()]
        heapq.heapify(distances_as_heap) # heap: [[0.0, '0'], [10.32, '8'], [95.01, '6'],...]
        # pop the first item that is not marked as visited
        while len(distances_as_heap) > 0: # '0' '1' '2' ...
            min_distance, index_at_min_dist = distances_as_heap.pop(0) 
            if visited_vertices_dict[index_at_min_dist] == False: break
        visited_vertices_dict[index_at_min_dist] = True
        # loop over the hash table keys and check if the registered distances[vertex] has a shorter connection
        for i in range(len(graph)):
            if visited_vertices_dict[i] == False and graph[index_at_min_dist][i] > 0 and \
            distances[index_at_min_dist] + graph[index_at_min_dist][i] < distances[i]: 
                distances[i] = distances[index_at_min_dist] + graph[index_at_min_dist][i]          
    return distances
print(djikstra_with_heapq(test_set, 3))
%timeit djikstra_with_heapq(test_set, 3)

{0: 26.46, 1: 33.959999999999994, 2: 45.62, 3: 0, 4: 1.88, 5: 18.689999999999998, 6: 22.919999999999998, 7: 14.77, 8: 33.42, 9: 26.64}
126 µs ± 4.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Adjacency list representation

In [7]:
import numpy as np
# turn the matrix into an adjacency list
adj_list = [[i, u, test_set[i][u]] for i in range(len(test_set)) for u in range(len(test_set))] 
# view the adjacency list
print(adj_list[5:15])


[[0, 5, 64.59], [0, 6, 43.76], [0, 7, 89.18], [0, 8, 96.37], [0, 9, 38.34], [1, 0, 79.17], [1, 1, inf], [1, 2, 56.8], [1, 3, 92.56], [1, 4, 7.1]]


### Bellman-Ford Algorithm for Graph Traversal

In [8]:
# Bellman-Ford Algorithm to traverse the test graph
# print(test_set) to view the matrix representation if needed

def graph_bellman_ford(adj_list, source): 
    # Initialize distances list from source vertex to other vertices
    dist = [np.inf] * len(test_set)
    # Make it acyclic to itself
    dist[source] = 0 
    # Loop over V-1 times and update the distance list where V is the number of vertices of the test_set (matrix)
    for i in range(len(test_set) - 1): 
        for u, v, w in adj_list: 
            if dist[u] != np.inf and w != 0 and dist[u] + w < dist[v]: 
                dist[v] = dist[u] + w 

    # Check for negative-weight cycles going over it one more time to see if there is any update.  
    for u, v, w in adj_list: 
        if dist[u] != np.inf and w != 0 and dist[u] + w < dist[v]: 
            print ("Graph contains negative weight cycle")
            return
    
    return dist
        
print(graph_bellman_ford(adj_list, 3)) 
# compare naive djikstra, improved djikstra, and bellman-ford
%timeit graph_bellman_ford(adj_list, 3) 
%timeit graph_traverse_DJ_path(test_set, 3)
# since toy matrix is very small the performance of the heap is not visiable
%timeit djikstra_with_heapq(test_set, 3) 

[26.46, 33.959999999999994, 39.82, 0, 1.88, 18.689999999999998, 22.919999999999998, 14.77, 33.42, 26.64]
675 µs ± 38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
120 µs ± 3.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
134 µs ± 3.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Notes:
Whether the graph is directed, cyclic, acyclic, or undirected does not matter. Dijkstra's algorithm would work. 

Dijkstra’s algorithm doesn’t work for graphs with negative weight edges. 

For graphs with negative weight edges, Bellman–Ford algorithm can be used.